In [153]:
import pandas as pd
import numpy as np
import statsmodels.formula.api as smf
from scipy import stats
import os

# Shutil is a recursive tool to delete file paths
# Note: Only needed if you plan to overwrite existing filepaths
import shutil

# Function: generateData
### Performs OLS on every edge and saves a, b into csv and json files 

> (bool) freight : Freight mode calculation on or off
(str) dataSourcePath : Source data file path (.xlsx)
(str) saveFolderName : Name of the folder to save the data (non-existing)

#### Note:  -1 is a placeholder for N/A values (self loops)

In [154]:
# This function will store the sampled data into multiple csv files.
def generateData (freight = True, dataSourcePath = 'Batch_Div27_Class8_2_Results_metrics_BEV.xlsx', saveFolderName = "ABData"):
    parent_dir = os.getcwd()
    savePath = os.path.join(parent_dir, saveFolderName)
    
    # If the folder already exists, remove the folder
    if os.path.exists(savePath):
        shutil.rmtree(savePath)
    os.makedirs(savePath)
    
    # Load and trim Data
    df = pd.read_excel(dataSourcePath, index_col=None)
    df = df.loc[:, ['Vehicle Model-none',\
            'Battery Energy Consumption per Mile-kWh/mi',\
            'Vehicle Static Mass-lbm',\
            'Cargo Mass-lbm']]
    
    # Getting information from the table
    vehicleTypes = pd.Series(data1['Vehicle Model-none'].values).unique()
    # Function to look for the number of nodes from a list of strings in the source file
    # Should also add columns to re-label the i and j-s into the main dataframe for lookup convenience later in this code
    def findInt(sentence_arr):
        running_max = 1
        length = len(sentence_arr)
        i_val = np.full(length, 0)
        j_val = np.full(length, 0)
        for index, sentence in enumerate(sentence_arr):
            digits = [int(i) for i in sentence if i.isdigit()] 
            running_max = max(digits[-1], digits[-2], running_max)
            i_val[index], j_val[index] = digits[-2], digits[-1]
        return running_max, i_val, j_val
    numNodes, i_val, j_val = findInt(data1["Cycle Source-none"].values)
    i_val, j_val = pd.Series(i_val), pd.Series(j_val)
    df['i'], df['j'] = i_val, j_val
    
    # Setup for labels
    cols = ["i_" + str(i) for i in range(1, numNodes+1)]
    rows = ["j_" + str(i) for i in range(1, numNodes+1)]
                   
    # Creating Vehicle File paths
    for v in vehicleTypes:
        workFrame = df[df["Vehicle Model-none"] == v]
        newFrame = pd.DataFrame(columns = cols, index = rows)

        # Iterate over node values.
        for i in range(1, numNodes + 1):
            for j in range(1, numNodes + 1):
                # This line below gives a frame of the cost of an edge
                data = workFrame.loc[((workFrame['i'] == i) & (workFrame['j'] == j))]

                # edge case if i = j or data Doesn't exist
                if i == j :
                    newFrame.iat[i-1, j-1] = -1
                    continue
                if data.size == 0:
                    newFrame.iat[i-1, j-1] = 0
                    continue
                    
                # Doing linear regression here
                if freight:
                    # Set training data to be Cargo data
                    train_data = data.loc[:, ['Cargo Mass-lbm']]*0.453592
                    train_data.columns = ['CargoMass_kg']
                    train_data['Energy_kwh_km'] = data.loc[:, ['Battery Energy Consumption per Mile-kWh/mi']]/1.60934
                    LR = smf.ols(formula='Energy_kwh_km ~ CargoMass_kg ',data=train_data).fit()
                else:
                    train_data = data.loc[:, ['Vehicle Static Mass-lbm']]*0.453592
                    train_data.columns = ['VehicleMass_kg']
                    train_data['Energy_kwh_km'] = data.loc[:, ['Battery Energy Consumption per Mile-kWh/mi']]/1.60934
                    LR = smf.ols(formula='Energy_kwh_km ~ VehicleMass_kg ',data=train_data).fit()
                
                # Assigning the calculated a, b params
                newFrame.iat[i - 1, j - 1] = (LR.params[0], LR.params[1])

        # Create the filePath
        tempPath2 = os.path.join(savePath, v + ".csv")
        tempPath3 = os.path.join(savePath, v + ".json")
        newFrame.to_csv(tempPath2)
        newFrame.to_json(tempPath3)

In [155]:
generateData(freight = True, dataSourcePath = 'Batch_Div27_Class8_2_Results_metrics_BEV.xlsx', saveFolderName = "ABData")

# Function: readData
 ### Reads the source folder generated by the  "generateData" function and stores it into a 5-dimensioned dictionary.
> Input:  
(str) saveFolderName: Folder Name of the folder created by generateData
  
Output: 
>a [ k ][ i ][ j ]  
b [ k ][ i ][ j ] 


>k : Vehiecle Type (string)    
i : Node  (int), index starts from 1  
j : Node  (int), index starts from 1  

#### Note:  -1 is a placeholder for N/A values (self loops) and -2 is used as padding (for lookup convenience) 

In [174]:
# Reads out the data and collects them into a dictionary.
def readDataCSV (saveFolderName = "ABData"):
    # Initialize some information
    path = os.path.join(os.getcwd(), saveFolderName)
    # If the folder doesn't exist, print error and return
    if not os.path.exists(path):
        print("Error, Source Folder Doesn't exist")
        return 
    VehiecleTypesPath = [ fi for fi in os.listdir(path) if fi.endswith(".json") ]
    VehiecleTypes = [i[:len(i) - 5] for i in VehiecleTypesPath]
    
    # Get the number of nodes brute forcedly
    filePath = os.path.join(path, VehiecleTypesPath[0])
    df = pd.read_json(filePath)
    numNodes = np.shape(df.iloc[:, 0].values)[0]
    
    # Return arr
    a = {}
    b = {}

    for ind, v in enumerate(VehiecleTypes):
        # initialize an empty array to store the days
        a[v] = np.full((numNodes + 1, numNodes + 1), -1, dtype = np.float64)
        b[v] = np.full((numNodes + 1, numNodes + 1), -1, dtype = np.float64)
        
        results = df.to_numpy().transpose()
        print(results)
        for i, temp in enumerate(results, 1):
            for j, ab in enumerate(temp, 1):
                print(i)
                if type(ab) == list:
                    print("flag: ", ab)
                    a[v][i][j], b[v][i][j] = ab[0], ab[1] 
                else:
                    a[v][i][j], b[v][i][j] = ab, ab
                    
    return a, b

In [175]:
a, b = readDataCSV()

[[-1 list([0.9624332969, 2.5122500000000002e-05]) 0 0 0 0]
 [list([0.9902330737, 2.74195e-05]) -1 list([1.0705120052, 2.59624e-05])
  0 0 0]
 [0 list([1.0570536054, 2.49329e-05]) -1 list([0.993294344, 2.65268e-05])
  0 0]
 [0 0 list([0.9670714249000001, 2.49375e-05]) -1 0 0]
 [0 0 0 0 -1 list([0.9951019945, 2.63563e-05])]
 [0 0 0 0 list([1.0104739061, 2.51175e-05]) -1]]
1
1
flag:  [0.9624332969, 2.5122500000000002e-05]
1
1
1
1
2
flag:  [0.9902330737, 2.74195e-05]
2
2
flag:  [1.0705120052, 2.59624e-05]
2
2
2
3
3
flag:  [1.0570536054, 2.49329e-05]
3
3
flag:  [0.993294344, 2.65268e-05]
3
3
4
4
4
flag:  [0.9670714249000001, 2.49375e-05]
4
4
4
5
5
5
5
5
5
flag:  [0.9951019945, 2.63563e-05]
6
6
6
6
6
flag:  [1.0104739061, 2.51175e-05]
6
[[-1 list([0.9624332969, 2.5122500000000002e-05]) 0 0 0 0]
 [list([0.9902330737, 2.74195e-05]) -1 list([1.0705120052, 2.59624e-05])
  0 0 0]
 [0 list([1.0570536054, 2.49329e-05]) -1 list([0.993294344, 2.65268e-05])
  0 0]
 [0 0 list([0.9670714249000001, 2.493

In [163]:
a

{'class8_bev_2021_daycab_regionalhaul_FU19': array([[-1.        , -1.        , -1.        , -1.        , -1.        ,
         -1.        , -1.        ],
        [-1.        ,  0.99023307,  0.        ,  0.        ,  0.        ,
          0.        , -1.        ],
        [-1.        , -1.        ,  1.05705361,  0.        ,  0.        ,
          0.        , -1.        ],
        [-1.        ,  1.07051201, -1.        ,  0.96707142,  0.        ,
          0.        , -1.        ],
        [-1.        ,  0.        ,  0.99329434, -1.        ,  0.        ,
          0.        , -1.        ],
        [-1.        ,  0.        ,  0.        ,  0.        , -1.        ,
          1.01047391, -1.        ],
        [-1.        ,  0.        ,  0.        ,  0.        ,  0.99510199,
         -1.        , -1.        ]]),
 'class8_bev_2021_low_sleeper_longhaul_FU19': array([[-1.        , -1.        , -1.        , -1.        , -1.        ,
         -1.        , -1.        ],
        [-1.        ,  0.9902

In [160]:
# Example lookup
a['class8_bev_2021_daycab_regionalhaul_FU19'][1][1]

0.9902330737